In [1]:
import os
import re
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

import shapely

import clima_anom as ca

import matplotlib.cbook
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import cartopy
import cartopy.feature as cf
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader

In [2]:
andes = list(shpreader.Reader('..'+os.sep+'mapping'+os.sep+'andes_1000.shp').geometries())

In [3]:
#Using IMERG 05x05 grid
file_in = '/mnt/Data/Data/IMERG/season/summer/IMERG_2000_12_2020_12_24h_05x05_summer.nc'
# file_in = 'E:'+os.sep+'Data'+os.sep+'IMERG'+os.sep+'season'+os.sep+'summer'+os.sep+'IMERG_2001_01_2020_12_24h_05x05_summer.nc'

data = ca.read_netcdf(file_in,0)
lat = data['lat'].data
lon = data['lon'].data

lon2d, lat2d = np.meshgrid(lon, lat)

nlat = len(lat)
nlon = len(lon)

res_y = lat[1]-lat[0]
res_x = lon[1]-lon[0]

del data

print(f'Latitude points   : {nlat}')
print(f'Longitude points  : {nlon}')
print(f'Spatial resolution: {res_x}x{res_y}')

Latitude points   : 120
Longitude points  : 110
Spatial resolution: 0.5x0.5


In [4]:
xmin, ymin, xmax, ymax= np.min(lon2d)-0.25,np.min(lat2d)+0.25,np.max(lon2d)-0.25,np.max(lat2d)+0.25

# how many cells across and down
cell_size_y=0.5
cell_size_x=0.5

# projection of the grid
crs = "+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs"
# create the cells in a loop
grid_cells = []
for x0 in np.arange(xmin, xmax+cell_size_x, cell_size_x ):
    for y0 in np.arange(ymin, ymax+cell_size_y, cell_size_y):
        # bounds
        x1 = x0+cell_size_x
        y1 = y0-cell_size_y
        grid_cells.append( shapely.geometry.box(x0, y0, x1, y1)  )

cell = gpd.GeoDataFrame(grid_cells, columns=['geometry'], crs="EPSG:4326")
cell = cell.reset_index()

In [5]:
product = 'kurpf'

In [6]:
path_in = os.sep+'mnt'+os.sep+'Data'+os.sep+'Data'+os.sep+'PF'+os.sep+product+'_SA'+os.sep+product+'_*.csv'
file_list = glob.glob(path_in)
file_list = sorted(file_list)
len_files = len(file_list)
print(f'Files Number: {len_files}')

Files Number: 85


In [7]:
print(file_list[0])
print(file_list[-1])

/mnt/Data/Data/PF/kurpf_SA/kurpf_201403_level2.csv
/mnt/Data/Data/PF/kurpf_SA/kurpf_202103_level2.csv


In [8]:
# Show columns
data_tmp = pd.read_csv(file_list[0])
data_tmp.head(2)

,lat,lon,YEAR,ORBIT,NPIXELS_20DBZ,NPIXELS_30DBZ,NPIXELS_40DBZ,NPIXELS_50DBZ,NPIXELS_20MM,NPIXELS_30MM,...,MIN85PCT,NLT250,NLT225,N37LT250,N37LT225,MAXHT20,MAXHT30,MAXHT40,NSTRAT_KU,NCONV_KU
0,14.503548,-47.985703,2014.0,148.0,2.0,0.0,0.0,0.0,0.0,0.0,...,283.450012,0.0,0.0,0.0,0.0,2.25,0.0,0.0,0.0,3.0
1,14.930592,-47.027515,2014.0,148.0,3.0,1.0,0.0,0.0,0.0,0.0,...,283.309998,0.0,0.0,0.0,0.0,2.38,2.0,0.0,0.0,3.0


In [9]:
# var = 'NPIXELS_40DBZ'
# var_name = '40dbz_GT_10km'
# val_min = 10

# var = 'NPIXELS_20DBZ'
# var_name = '20dbz_GT_10km'
# val_min = 10

# var = 'MAXHT40'
# var_name = 'MAXHT40_GT_10km'
# val_min = 10

# var = 'MAXHT20'
# var_name = 'MAXHT20_GT_10km'
# val_min = 10

# var = 'NPIXELS_20MM'
# var_name = 'NPIXELS_GT_20MM'
# val_min = 0

var = 'NPIXELS_30MM'
var_name = 'NPIXELS_GT_30MM'
val_min = 0

# var = 'NCONV_KU'
# var_name = 'NCONV'
# val_min = 0

# var = 'MIN85PCT'
# var_name = 'MIN85PCT_LT_225K'
# val_min = 225

In [10]:
# https://pygis.io/docs/e_summarize_vector.html
pf_count = np.zeros([len_files,nlat,nlon])

for t in range(len_files):

    file_in = file_list[t]
    name_out = file_in.split('/')[-1]
    print(name_out)

    data = pd.read_csv(file_in)
    data_cut = data[data[var] > val_min].reset_index(drop=True)

    PF_GPD = gpd.GeoDataFrame(geometry=gpd.points_from_xy(data_cut['lon'],data_cut['lat'], crs="EPSG:4326"))

    wells_cell = gpd.sjoin(PF_GPD, cell, how = "inner", predicate = "intersects")
    wells_cell = wells_cell.drop_duplicates(subset = ['index']).reset_index(drop = True)
    count_field = "Count"
    wells_cell[count_field] = 1
    wells_cell = wells_cell.groupby('index').agg({count_field:'sum'})
    bay_area_grid = cell.merge(wells_cell, on = 'index', how = "left")
    bay_area_grid[count_field] = bay_area_grid[count_field].fillna(0)
    bay_area_grid[count_field] = bay_area_grid[count_field].astype(int)

    data_out = bay_area_grid['Count'].values.reshape(nlon, nlat).T

    pf_count[t,:,:] = data_out

kurpf_201403_level2.csv
kurpf_201404_level2.csv
kurpf_201405_level2.csv
kurpf_201406_level2.csv
kurpf_201407_level2.csv
kurpf_201408_level2.csv
kurpf_201409_level2.csv
kurpf_201410_level2.csv
kurpf_201411_level2.csv
kurpf_201412_level2.csv
kurpf_201501_level2.csv
kurpf_201502_level2.csv
kurpf_201503_level2.csv
kurpf_201504_level2.csv
kurpf_201505_level2.csv
kurpf_201506_level2.csv
kurpf_201507_level2.csv
kurpf_201508_level2.csv
kurpf_201509_level2.csv
kurpf_201510_level2.csv
kurpf_201511_level2.csv
kurpf_201512_level2.csv
kurpf_201601_level2.csv
kurpf_201602_level2.csv
kurpf_201603_level2.csv
kurpf_201604_level2.csv
kurpf_201605_level2.csv
kurpf_201606_level2.csv
kurpf_201607_level2.csv
kurpf_201608_level2.csv
kurpf_201609_level2.csv
kurpf_201610_level2.csv
kurpf_201611_level2.csv
kurpf_201612_level2.csv
kurpf_201701_level2.csv
kurpf_201702_level2.csv
kurpf_201703_level2.csv
kurpf_201704_level2.csv
kurpf_201705_level2.csv
kurpf_201706_level2.csv
kurpf_201707_level2.csv
kurpf_201708_lev

In [11]:
print(f'pf count shape: {pf_count.shape}')
info = {'file': '/mnt/Data/Data/PF/'+product+'_SA_count/'+product+'_SA_count_'+var_name+'.nc',
        'title': 'Number of events to '+ product +' for '+ var_name +' for IMERG grid 0.5x0.5',
        'year_start':2014,'month_start':3,'day_start':1,'hour_start':0,'minute_start':0,
        'year_end':2021,'month_end':3,'day_end':1,'hour_end':23,'minute_end':59,
        'time_frequency': 'Monthly',
        'time_interval':1,
        'var_name': 'count',
        'var_units': 'adim'}
print(info['file'])

pf count shape: (85, 120, 110)
/mnt/Data/Data/PF/kurpf_SA_count/kurpf_SA_count_NPIXELS_GT_30MM.nc


In [12]:
ca.create_netcdf(info,pf_count,lat,lon)

Time Start: 2014-03-01 00:00:00
Time End: 2021-03-01 00:00:00
Time Frequency: Monthly
Time Lenght: 85

File created in:  /mnt/Data/Data/PF/kurpf_SA_count/kurpf_SA_count_NPIXELS_GT_30MM.nc
File title:  Number of events to kurpf for NPIXELS_GT_30MM for IMERG grid 0.5x0.5
var title:  count
var units:  adim
